In [1]:
import numpy as np
import pandas as pd
import math

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.exporter.xes import exporter as xes_exporter

In [124]:
log = xes_importer.apply('FMP_no_appeal.xes')

In [125]:
df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

In [128]:
traces = df.groupby('case:concept:name')['concept:name'].apply(np.asarray)

In [131]:
last_event = traces.apply(lambda x : x[-1])
last_event.name = 'lastEvent'
df = df.join(last_event,on='case:concept:name')

In [134]:
number_cases = len(df['case:concept:name'].unique())
number_credit_collections = len(df[df['lastEvent']=='Send for Credit Collection']['case:concept:name'].unique())
df_no_collect = df[df['lastEvent']!='Send for Credit Collection']
number_credit_collections

58601

In [140]:
toPay = (df_no_collect.groupby('case:concept:name')['amount'].max() +
         df_no_collect.groupby('case:concept:name')['expense'].sum())
payed = df_no_collect.groupby('case:concept:name')['totalPaymentAmount'].max()
balance = toPay - payed


In [151]:

exact_payment = (balance[balance==0].count() + number_credit_collections )/ number_cases

over_payment = balance[balance<0].count() / number_cases

no_payment = payed[payed==0].count() / number_cases

some_payment = (balance[balance>0].count() -  payed[payed==0].count()) / number_cases

total_deficit = balance.sum()

total_deficit_cleaned = balance[balance>0].sum()

#sanity_check:
print(exact_payment + over_payment + no_payment + some_payment == 1)

print(exact_payment)
print(over_payment)
print(no_payment)
print(some_payment)
print(total_deficit)
print(total_deficit_cleaned)

True
0.8026423140473203
0.010441734027163592
0.139794456213963
0.047121495711553094
1438950.6
1459775.5


# Playground


In [127]:
#values that total Payment takes for non Payment events
print(df[df['concept:name']!='Payment']['totalPaymentAmount'].unique())

#values that payment amount takes for non Payment events
print(df[df['concept:name']!='Payment']['paymentAmount'].unique())

[ 0. nan]
[nan]


In [130]:
traces.apply(lambda x : 'Send for Credit Collection' in x[:-1]).any()
#If Send for credit collection appears it is the last event

False

In [152]:
df.head()

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,lastEvent
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-24 00:00:00+02:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,Send Fine
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-05 00:00:00+01:00,NaN,NaN,A1,11.0,NaN,NaN,NaN,Send Fine
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-02 00:00:00+02:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,Send for Credit Collection
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-12 00:00:00+01:00,NaN,NaN,A100,11.0,NaN,NaN,NaN,Send for Credit Collection
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-15 00:00:00+01:00,NaN,NaN,A100,NaN,P,P,NaN,Send for Credit Collection
